In [1]:
import pandas as pd
import os

In [2]:
base_dir = './SisFall_dataset-2/'
save_dir = './training_data/fall'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

valid_person_numbers = list(range(6, 11))

# txt파일을 save_dir에 csv형식으로 저장
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    
    if os.path.isdir(folder_path): 
        for file_name in os.listdir(folder_path):
            if file_name.startswith(('F08', 'F09', 'F10', 'F11')):
                parts = file_name.split('_')
                
                person_number = int(parts[1][2:])
                if person_number in valid_person_numbers:
                    file_path = os.path.join(folder_path, file_name)
                    data = pd.read_csv(file_path, delimiter='\t', header=None)
                    
                    csv_file_name = file_name.replace('.txt', '.csv')
                    save_path = os.path.join(save_dir, csv_file_name)
                    data.to_csv(save_path, index=False, header=False)

In [8]:
# 변환한 csv 파일 정제
def clean_data(data_str):
    data_str = data_str.replace(";", "").replace('"', "").strip()
    data_list = [float(num.strip()) for num in data_str.split(",") if num.strip()]
    
    return data_list

# 데이터 전처리
def convert_data(file_path, output_dir):
    # 파일 읽기
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # 각 줄을 처리하여 데이터프레임으로 변환
    cleaned_data = [clean_data(line) for line in lines]
    data = pd.DataFrame(cleaned_data, columns=['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'other1', 'other2', 'other3'])

    # 필요한 열만 선택 (가속도와 자이로 데이터를 사용하는 경우)
    data = data.iloc[:, :6]

    # 가속도 데이터 변환
    adxl345_range_value = 16  # ±16g
    adxl345_resolution = 13  # 13 bits
    adxl345_conversion_factor = (2 * adxl345_range_value) / (2 ** adxl345_resolution) 

    # 스마트워치 가속도 범위
    watch_acc_max_range = 78.4532

    # 스케일링
    data['accX'] = (data['accX'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)
    data['accY'] = (data['accY'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)
    data['accZ'] = (data['accZ'] * adxl345_conversion_factor * (watch_acc_max_range / adxl345_range_value)).round(2)

    # 각속도 데이터 변환
    itg3200_range_value = 2000  # ±2000 °/s
    itg3200_resolution = 16  # 16 bits
    itg3200_conversion_factor = (2 * itg3200_range_value) / (2 ** itg3200_resolution) 

    # 스마트워치 자이로스코프 범위
    watch_gyro_max_range = 17.453642

    # 스케일링
    data['gyroX'] = (data['gyroX'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)
    data['gyroY'] = (data['gyroY'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)
    data['gyroZ'] = (data['gyroZ'] * itg3200_conversion_factor * (watch_gyro_max_range / itg3200_range_value)).round(2)

    output_file_path = os.path.join(output_dir, os.path.basename(file_path))
    data.to_csv(output_file_path, index=False)
    
# csv 형식으로 저장
def process_directory(input_dir, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith('.csv'):
                file_path = os.path.join(root, file)
                convert_data(file_path, output_dir)

# 적용할 디렉토리 경로
input_dir = './training_data/prev'
output_dir = './training_data/fall'

# 디렉토리 처리
process_directory(_dir, output_dir)